# <span style="color:darkblue"> Lecture 23 - Practicing Time and Plots  </span>

<font size = "5">

In  this lecture we will practice working with time in Python

- There will be a quiz on this lecture


# <span style="color:darkblue"> I. Review of time commands </span>

<font size = "5">

Line plot: $\qquad \qquad \qquad \quad \ $ ``` plt.plot() ```

Parse string column to date: $\ $``` pd.to_datetime(data_column,format)```

Date to string:  $\qquad \qquad \quad \ $ ```.dt.strftime(format)```

Group by time period:
$\qquad \ \ $ ``` pd.Grouper(key='date', freq='m')```

Include group as column: $\quad$ ```.reset_index()```

Difference between periods: $\ $``` .diff() ```

Lag a periods: $\qquad \qquad \quad \ $ ``` .shift(1) ```

Between two periods (bool): $\ $ ```.between("date1","date2")```

Convert wide to long: $\qquad \quad $ ``` pd.melt(...) ```

Convert long to wide: $\qquad \quad $ ``` pd.pivot(...) ```

## <span style="color:darkblue"> II. Quiz Structure </span>

<font size = "5">

The day of the quiz I will ...
- Provide a dataset with information
- Give more specific instructions.
- Below, you will see the type of questions that will be asked.
- The idea is for you to apply known concepts to new data
- You have 50 minutes to complete the assignment

Questions

(exact wording may change in quiz, but exercise will be very similar)

The quiz will use a time series dataset



<font size = "5">

(a) Import a dataset, parse date column using wildcards <br>
$\quad$ (see Lecture 21), and sort dataset

(b) Line plots from multiple columns with a legend

(c) Compute growth rates for a time series

(d) Flag a subset of the data and produce a shaded plot <br>
$\quad$ as in Lecture 22

(e) Compute aggregate dataset by time period

(f) Convert a dataset to long format


<font size = "5">

Convert to string (a)


- A wildcard ```%``` is used to denote date formats
- Useful when working with text data

$\quad$ <img src="figures/wildcards_fig1.png" alt="drawing" width="300"/>

In [ ]:
#  a)
# import dataset, 

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import date, time, datetime

financial = pd.read_csv("data_raw/financial.csv")

#parse date column using wildcars

financial["date"] = pd.to_datetime(financial["date_str"])
financial["date_test"] = financial["date"].dt.strftime("%A, %B %d, %Y")

display(financial["date_test"])

#sort dataset

week_config = pd.Grouper(key = "date",freq = "w")

weekly_data = financial.groupby(week_config).agg(std_weekly = ("sp500","std")).reset_index()
c


In [ ]:

# (b) Line plots from multiple columns with a legend

# Enter the x-axis column and y-axis columns you 
# wish to include. Specify the x-axis column with "set_index()"
# This applies to any line plot, with or without dates
# The legend is the box with the name of the lines
# If you drop the ".legend()" command this will assign
# the default column names to the legend.


financial[["date","sp500","djia"]].set_index("date").plot()
plt.xlabel("Time")
plt.ylabel("Value of Index Funds")
plt.legend(["S&P 500","Dow Jones"])


In [ ]:

#(c) Compute growth rates for a time series

financial["diff_sp500"] = financial["sp500"].diff()

financial["diff_date"]  = financial["date"]-financial["date"].shift(1)
financial["count_days"] = financial["diff_date"].dt.days


financial["lag_sp500"]    = financial["sp500"].shift(1)
financial["growth_sp500"] = financial["diff_sp500"]* 100/financial["lag_sp500"]


In [ ]:

#(d) Flag a subset of the data 

subset_before  = financial.query('date >= "2019-01-01" ')
subset_after   = financial.query('date <= "2020-01-01" ')

subset_between = financial.query('date.between("2020-03-01","2020-05-01")')
financial["bool_period"]  = financial["date"].between("2020-03-01","2020-05-01")
financial["bool_example"] = financial["growth_sp500"] > 5

# produce a shaded plot lec22
# Create a line plot
plt.plot("date", "growth_sp500", data = financial)
plt.xlabel("Time")
plt.ylabel("Daily percentage change ")
plt.title("The S&P 500 during the start of COVID")

# Add a shaded region wth a rectangle
# "x" is the x-coordinates, "y1" and "y2" are the lower
# and upper bounds of the rectangle. We can set this
# to be the minimum and meximum of the outcome.
# we use "where" to test a logical condition

vec_y = financial["growth_sp500"]
plt.fill_between(x= financial["date"],
                 y1 = vec_y.min(),
                 y2 = vec_y.max(),
                 where = financial["bool_period"],
                 alpha = 0.2,color = "red")

In [ ]:

#(e) Compute aggregate dataset by time period

financial_long2=pd.melt(financial,
                     var_name="var_name",
                     value_name = "value_name",
                     id_vars="date",
                     value_vars=['growth_sp500','growth_djia'])
display(financial_long2)

example5 = pd.read_sql(text("SELECT AVG(points) as mean_points \
                             FROM results_sql \
                             GROUP BY time ;"),con)


In [ ]:
#(f) Convert a dataset to long format


financial_long2=pd.melt(financial,
                     var_name="var_name",
                     value_name = "value_name",
                     id_vars="date",
                     value_vars=['growth_sp500','growth_djia'])
display(financial_long2)


financial_wide = (pd.pivot(financial_long, #melt - horizontal -> vertical
                           index = 'date',
                           columns = 'portfolio_type',
                           values =  'portfolio_value'))
#turning long to wide
# If you also want the index to be part of the dataset add
# ".reset_index()" to the end of the previous command